In [ ]:
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation, AutoConfig
from PIL import Image
import requests

image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512",)
config=AutoConfig.from_pretrained("nvidia/mit-b0")
config.strides=[16,4,2,2]
model = SegformerForSemanticSegmentation(config).from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512",
                                                    config=config,
                                                    ignore_mismatched_sizes=True,
                                                    )

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = image_processor(images=image, return_tensors="pt")
inputs["pixel_values"]=inputs["pixel_values"][:,:,:224,:224]
outputs = model(**inputs)
logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
list(logits.shape)

In [ ]:
from transformers import SegformerConfig, AutoConfig

In [ ]:
SegformerConfig.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

In [ ]:
AutoConfig.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512").id2label

In [ ]:
SegformerConfig()

In [ ]:
import pandas as pd
name_gvkey=pd.read_csv("name_gvkey.csv")
unique_names=pd.read_csv("unique_names.csv")

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel, AutoTokenizer

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

model.cuda()

In [ ]:
import tqdm
import torch

In [ ]:
# get text features for all companies in name_gvkey

text_features_ref=[]
with torch.no_grad():
    for template in ["company {}", "company name {}", "{} Inc"]:
        text_features_ref_template=[]
        for i in tqdm.tqdm(range(0, len(name_gvkey["conml"]), 64)):
            name_gvkey_list=name_gvkey["conml"][i:min(i+64,len(name_gvkey["conml"]))]
            prompt=[f"company {company_name}" for company_name in name_gvkey_list]
            inputs_ref_=tokenizer(prompt, padding=True,  return_tensors="pt")
            text_features_ref_ = model.get_text_features(inputs_ref_["input_ids"].cuda()) 
            text_features_ref_template.append(text_features_ref_)
        text_features_ref_template=torch.vstack(text_features_ref_template)
        text_features_ref.append(text_features_ref_template)
    text_features_ref=torch.vstack([i.unsqueeze(0) for i in text_features_ref])        


In [ ]:
text_features_data=[]
with torch.no_grad():
    for template in ["company {}", "company name {}", "{} Inc"]:
        text_features_data_template=[]
        for i in tqdm.tqdm(range(0, len(unique_names["company"]), 64)):
            company_list=unique_names["company"][i:min(i+64,len(unique_names["company"]))]
            prompt=[f"company {company_name}" for company_name in company_list]
            inputs_data_=tokenizer(prompt, padding=True,  return_tensors="pt")
            text_features_data_ = model.get_text_features(inputs_data_["input_ids"].cuda()) 
            text_features_data_template.append(text_features_data_)
        text_features_data_template=torch.vstack(text_features_data_template)
        text_features_data.append(text_features_data_template)
    text_features_data=torch.vstack([i.unsqueeze(0) for i in text_features_data])        

In [ ]:
text_features_ref=torch.vstack([i.unsqueeze(0) for i in text_features_ref])

In [ ]:
with torch.no_grad():
    for idx,company_name in enumerate(unique_names["company"]):
        inputs1 = tokenizer([f"Company {company_name}"], padding=True, return_tensors="pt")
        text_features1 = model.get_text_features(inputs1["input_ids"].to("cuda:0"))   

        sim=(text_features1 / text_features1.norm(dim=1, keepdim=True))@\
        (text_features_ref / text_features_ref.norm(dim=1, keepdim=True)).T
        print((company_name,pd.Series(sim[0].detach().cpu().numpy(),index=name_gvkey["conml"]).sort_values(ascending=False).iloc[:5].index))
        if idx==100:
            break

In [ ]:
inputs_ref = tokenizer([f"company {i}" for i in name_gvkey["conml"]], padding=True, return_tensors="pt")
text_features_ref = model.get_text_features(inputs_ref["input_ids"]) 

In [ ]:
name_gvkey["conml"][name_gvkey["conml"].str.lower().str.contains("yahoo")]

In [ ]:
model.get_text_features(processor(text=["a photo of a cat", "a photo of a dog"])["input_ids"])

In [ ]:

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [ ]:
model.

In [ ]:
text1 = "Uber"
encoded_input1 = tokenizer(text1, return_tensors='pt')
output1 = model(**encoded_input1,output_hidden_states=True)

In [ ]:
text2 = ["Uber Technology", "AA tech", 'facebook']
encoded_input2 = tokenizer(text2, return_tensors='pt', padding=True)
output2 = model(**encoded_input2,output_hidden_states=True)

In [ ]:
(output1["hidden_states"][-1]).shape

In [ ]:
(output2["hidden_states"][-1]).shape

In [ ]:
(output1["pooler_output"]/output1["pooler_output"].norm(dim=1, keepdim=True))\
@(output2["pooler_output"]/output2["pooler_output"].norm(dim=1, keepdim=True)).T

In [ ]:
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
import requests

image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
list(logits.shape)

In [ ]:
image

In [ ]:
logits.shape

In [ ]:
inputs["pixel_values"].shape

In [ ]:
from transformers import (
    MODEL_FOR_IMAGE_CLASSIFICATION_MAPPING,
    AutoConfig,
    AutoImageProcessor,
    AutoModelForImageClassification,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [ ]:
surrogate_config = AutoConfig.from_pretrained(
    "logs/vitbase_imagenette_surrogate",
    num_labels=10,
)

In [ ]:

from models import SurrogateForImageClassificationConfig

surrogate_for_image_classification_config = SurrogateForImageClassificationConfig(
    surrogate_pretrained_model_name_or_path="logs/vitbase_imagenette_surrogate",
    surrogate_config=surrogate_config,
)

In [ ]:
from models import SurrogateForImageClassification

surrogate = SurrogateForImageClassification(
    config=surrogate_for_image_classification_config,
)

In [ ]:
SurrogateForImageClassification.from_pretrained("logs/vitbase_imagenette_surrogate")

In [ ]:
explainer_config = AutoConfig.from_pretrained(
    "logs/vitbase_imagenette_surrogate"
)

In [ ]:
    
from models import ExplainerForImageClassificationConfig

explainer_for_image_classification_config = ExplainerForImageClassificationConfig(
    surrogate_pretrained_model_name_or_path="logs/vitbase_imagenette_surrogate",
    surrogate_config=surrogate_for_image_classification_config,
    explainer_pretrained_model_name_or_path="logs/vitbase_imagenette_surrogate",
    explainer_config=explainer_config,
)

In [ ]:
    
from models import ExplainerForImageClassification

explainer = ExplainerForImageClassification(
    config=explainer_for_image_classification_config,
)

In [ ]:
from transformers import PretrainedConfig
from typing import List


class ResnetConfig(PretrainedConfig):
    model_type = "resnet"

    def __init__(
        self,
        stem_type,
        block_type="bottleneck",
        layers: List[int] = [3, 4, 6, 3],
        num_classes: int = 1000,
        input_channels: int = 3,
        cardinality: int = 1,
        base_width: int = 64,
        stem_width: int = 64,
        
        avg_down: bool = False,
        **kwargs,
    ):
        if block_type not in ["basic", "bottleneck"]:
            raise ValueError(f"`block_type` must be 'basic' or bottleneck', got {block_type}.")
        if stem_type not in ["", "deep", "deep-tiered"]:
            raise ValueError(f"`stem_type` must be '', 'deep' or 'deep-tiered', got {stem_type}.")

        self.block_type = block_type
        self.layers = layers
        self.num_classes = num_classes
        self.input_channels = input_channels
        self.cardinality = cardinality
        self.base_width = base_width
        self.stem_width = stem_width
        self.stem_type = stem_type
        self.avg_down = avg_down
        super().__init__(**kwargs)

In [ ]:
resnet50d_config = ResnetConfig(block_type="bottleneck", stem_width=32, stem_type="deep", avg_down=True)
resnet50d_config.save_pretrained("custom-resnet")

In [ ]:
resnet50d_config.__dict__.get("problem_type", None)

"       ".join([i for i in dir(resnet50d_config)])

In [ ]:
resnet50d_config = ResnetConfig.from_pretrained("custom-resnet")

In [ ]:
# Load model directly
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from transformers import AutoConfig, AutoModel

In [ ]:
from transformers import AutoModelForDepthEstimation
AutoModelForDepthEstimation.from_pretrained("google/vit-base-patch16-224")

In [ ]:
# Load model directly
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from transformers import AutoConfig, AutoModel

extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
AutoModelForImageClassification.from_pretrained("microsoft/resnet-50")

In [ ]:
AutoModel.from_pretrained("microsoft/resnet-50")

In [ ]:
b = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

In [ ]:
b

In [ ]:
a=AutoModel.from_pretrained("google/vit-base-patch16-224")

In [ ]:
a

In [ ]:
class MyClass:
    def method(self):
        print("Original method")

def new_method(self):
    print("New method for this instance")

obj = MyClass()
obj.method()  # This will print: Original method

# Change method only for this instance
obj.method = new_method.__get__(obj, MyClass)
obj.method()  # This will print: New method for this instance

In [ ]:
new_method.__get__?

In [ ]:
from transformers import (
    MODEL_FOR_IMAGE_CLASSIFICATION_MAPPING,
    AutoConfig,
    AutoImageProcessor,
    AutoModelForImageClassification,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [ ]:
classifier_config = AutoConfig.from_pretrained("logs/resnet50_imagenette/")

In [ ]:
classifier = AutoModelForImageClassification.from_pretrained(
    "logs/resnet50_imagenette/")

In [ ]:
classifier

In [ ]:
classifier_config

In [ ]:
classifier_config = AutoConfig.from_pretrained(
    classifier_args.classifier_config_name
    or classifier_args.classifier_model_name_or_path,
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label,
    finetuning_task="image-classification",
    cache_dir=classifier_args.classifier_cache_dir,
    revision=classifier_args.classifier_model_revision,
    token=other_args.token,
)
classifier = AutoModelForImageClassification.from_pretrained(
    classifier_args.classifier_model_name_or_path,
    from_tf=bool(".ckpt" in classifier_args.classifier_model_name_or_path),
    config=classifier_config,
    cache_dir=classifier_args.classifier_cache_dir,
    revision=classifier_args.classifier_model_revision,
    token=other_args.token,
    ignore_mismatched_sizes=classifier_args.classifier_ignore_mismatched_sizes,
)